# Import

In [102]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config IPCompleter.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
os.listdir(os.getcwd())

['.~lock.ADaM spec.xlsx#',
 'ADSL.xlsx',
 'ADaM spec.xlsx',
 'adae_construction.qmd',
 'adams construction.Rproj',
 'dm_lecture.qmd',
 'images',
 'sdtm_files',
 '.ipynb_checkpoints',
 'adae.ipynb']

# Data

In [104]:
ADSL = pd.read_excel('ADSL.xlsx')
ADSL

,STUDYID,SUBJID,USUBJID,SITEID,RANDID,TRTSEQP,TRTSEQPN,TRT01P,TRT01PN,TRT02P,...,SMOKDC,ALCCDC,DOREAS,ECREAS,DOREASPP,DOREASSAF,RANDFL,PPROTFL,SAFFL,COMPLFL
0,XXXXX-XX-XX,9005,XXXXX-XX-XX-09005,9,5.0,RT,2.0,drug2,2.0,drug1,...,Нет,Употреблял ранее,NaN,NaN,NaN,NaN,Y,Y,Y,Y
1,XXXXX-XX-XX,9006,XXXXX-XX-XX-09006,9,6.0,RT,2.0,drug2,2.0,drug1,...,Нет,Да,NaN,NaN,NaN,NaN,Y,Y,Y,Y
2,XXXXX-XX-XX,9007,XXXXX-XX-XX-09007,9,7.0,TR,1.0,drug1,1.0,drug2,...,Нет,Нет,NaN,NaN,NaN,NaN,Y,Y,Y,Y
3,XXXXX-XX-XX,9008,XXXXX-XX-XX-09008,9,8.0,RT,2.0,drug2,2.0,drug1,...,Нет,Нет,NaN,Случай НЯ,Выбыл из исследования,NaN,Y,N,Y,N
4,XXXXX-XX-XX,9009,XXXXX-XX-XX-09009,9,NaN,NaN,NaN,NaN,NaN,NaN,...,Не известно,Не известно,Несоответствие критериям включения/невключения,NaN,Выбыл из исследования,NaN,N,N,N,N


In [105]:
ADSL.index = ADSL.SUBJID.to_list()

from datetime import datetime
def convert_to_date(date_str, fmt):
    if type(date_str)!=float:
        x = datetime.strptime(date_str, fmt)
        return x
    else:
        return None
ADSL['AP01SDT'] = ADSL['AP01SDT'].apply(lambda x: convert_to_date(x, '%d.%m.%Y'))
ADSL['AP02SDT'] = ADSL['AP02SDT'].apply(lambda x: convert_to_date(x, '%d.%m.%Y'))
ADSL['AP01EDT'] = ADSL['AP01EDT'].apply(lambda x: convert_to_date(x, '%d.%m.%Y'))
ADSL['AP02EDT'] = ADSL['AP02EDT'].apply(lambda x: convert_to_date(x, '%d.%m.%Y'))

In [106]:
ADSL = ADSL[ADSL.RANDFL=='Y']

In [107]:
adae = {}

In [108]:
AE = pd.read_excel('/home/jovyan/projects/Biostat/BioStat_2023/Data_control/adams_construction/sdtm_files/AE_ADVERSEEVENTS.xlsx')
AE['AESTDTC'] = AE['AESTDTC'].apply(lambda x: convert_to_date(x, '%Y-%m-%d'))
AE['AEENDTC'] = AE['AEENDTC'].apply(lambda x: convert_to_date(x, '%Y-%m-%d'))
SUPPAE = pd.read_excel('/home/jovyan/projects/Biostat/BioStat_2023/Data_control/adams_construction/sdtm_files/SUPPAE_ADVERSEEVENTS.xlsx')

In [109]:
for col in ['STUDYID',
 'SUBJID',
 'USUBJID',
 'SITEID',
 'TRTSEQP',
 'TRTSEQPN',
 'AP01SDT',
 'AP01EDT',
 'AP02SDT',
 'AP02EDT',
           'SAFFL',
'AGE',
'SEX',
'WEIGHT',
'RACE'
]:
     adsl_col = 'WEIGHTBL' if col == 'WEIGHT' else col
     adae[col] = AE['SUBJID'].map(lambda x: ADSL.loc[x,adsl_col])

In [110]:
adae['AREPIOD'] = {}
combined = AE.merge(ADSL, on='SUBJID', how='left')
for i in combined.index:
    period = 0
    d = combined.loc[i]
    ae_st = d.loc['AESTDTC']
    p1_st = d.loc['AP01SDT']
    p1_end = d.loc['AP01EDT']
    p2_st = d.loc['AP02SDT']
    p2_end = d.loc['AP02EDT']
    if p1_st and p1_end:
        if ae_st>=p1_st and ae_st<=p1_end:
            period = 1
        elif ae_st<p1_st:
            print(d.SUBJID)
            period = None
    else:
        period = None
    if p2_st and p2_end:
        if ae_st>=p2_st and ae_st<=p2_end:
            period = 2
    else:
        period = None
    adae['AREPIOD'][i] = period
adae['AREPIOD'] = pd.Series(adae['AREPIOD'])
adae['AREPIOD']

9008


0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     2.0
6     2.0
7     2.0
8     NaN
9     1.0
10    1.0
dtype: float64

In [111]:
adae['APERIODC'] = adae['AREPIOD'].map(lambda x: f"Период {x}" if x==1 or x==2 else np.nan)
adae['APERIODC'] 

0     Период 1.0
1     Период 1.0
2     Период 1.0
3     Период 1.0
4     Период 1.0
5     Период 2.0
6     Период 2.0
7     Период 2.0
8            NaN
9     Период 1.0
10    Период 1.0
dtype: object

In [112]:
adae['TRTP'] = {}
for i in adae['AREPIOD'].index:
    if  adae['AREPIOD'].loc[i] ==1:
        adae['TRTP'][i] = ADSL[ADSL.SUBJID==adae['SUBJID'].loc[i]].TRT01P.iloc[0]
    elif adae['AREPIOD'].loc[i] ==2:
        adae['TRTP'][i] = ADSL[ADSL.SUBJID==adae['SUBJID'].loc[i]].TRT02P.iloc[0]
    else:
        adae['TRTP'][i] = np.nan
adae['TRTP'] = pd.Series(adae['TRTP'])
adae['TRTP']

0     drug2
1     drug2
2     drug2
3     drug2
4     drug2
5     drug2
6     drug2
7     drug2
8       NaN
9     drug2
10    drug2
dtype: object

In [113]:
adae['TRTPN'] = {}
for i in adae['AREPIOD'].index:
    if  adae['AREPIOD'].loc[i] ==1:
        adae['TRTPN'][i] = ADSL[ADSL.SUBJID==adae['SUBJID'].loc[i]].TRT01PN.iloc[0]
    elif adae['AREPIOD'].loc[i] ==2:
        adae['TRTPN'][i] = ADSL[ADSL.SUBJID==adae['SUBJID'].loc[i]].TRT02PN.iloc[0]
    else:
        adae['TRTPN'][i] = np.nan
adae['TRTPN'] = pd.Series(adae['TRTPN'])
adae['TRTPN']

0     2.0
1     2.0
2     2.0
3     2.0
4     2.0
5     2.0
6     2.0
7     2.0
8     NaN
9     2.0
10    2.0
dtype: float64

In [114]:
adae['AESEQ'] = AE.AESEQ 
adae['AESEQ'] 

0     1
1     2
2     1
3     1
4     2
5     1
6     2
7     1
8     1
9     1
10    1
Name: AESEQ, dtype: int64

In [115]:
adae['AETERM'] = AE.AETERM 
adae['AETERM']

0     TRANSITORY BURNING IN SITE OF APPLICATION
1                      APPLICATION SITE BURNING
2              BURNING SENSATION WHEN URINATING
3                                URINATION PAIN
4                                URINATION PAIN
5                             PAINFUL URINATION
6                             PAINFUL URINATION
7                      INCREASED BLOOD PRESSURE
8                              NASAL CONGESTION
9                        RETROGRADE EJACULATION
10        SLUGGISH URINATION,  INCOMPLETE EMPTY
Name: AETERM, dtype: object

In [116]:
AE.AESTDTC

0    2023-06-28
1    2023-06-29
2    2023-06-29
3    2023-06-29
4    2023-06-30
5    2023-07-05
6    2023-07-06
7    2023-07-06
8    2023-06-27
9    2023-06-28
10   2023-06-29
Name: AESTDTC, dtype: datetime64[ns]

In [117]:
adae['ASTDT'] = AE.AESTDTC.dt.strftime('%d.%m.%Y')
adae['AENDT'] = AE.AEENDTC.dt.strftime('%d.%m.%Y')
adae['AENDT']

0     28.06.2023
1     29.06.2023
2     29.06.2023
3     29.06.2023
4     30.06.2023
5     05.07.2023
6     06.07.2023
7     06.07.2023
8     27.06.2023
9     28.06.2023
10    29.06.2023
Name: AEENDTC, dtype: object

In [118]:
terms_translation = pd.read_excel('/home/jovyan/projects/Biostat/BioStat_2023/Data_control/adams_construction/sdtm_files/terms_translation.xlsx')
terms_translation

,PT,PTT,SOC,SOCT
0,Administration site irritation,Раздражение в месте введения ...,General disorders and administration site cond...,Общие нарушения и реакции в месте введения
1,Dysuria,Дизурия,Renal and urinary disorders,Нарушения со стороны почек и мочевыводящих путей
2,Hypertension,Гипертензия,Vascular disorders,Нарушения со стороны сосудов
3,Nasal congestion,Заложенность носа,"Respiratory, thoracic and mediastinal disorders","Нарушения со стороны дыхательной системы, орга..."
4,Retrograde ejaculation,Ретроградная эякуляция,Reproductive system and breast disorders,Нарушения со стороны репродуктивной системы и ...
5,Urine flow decreased,Замедленная струя мочи,Renal and urinary disorders,Нарушения со стороны почек и мочевыводящих путей


In [119]:
terms_translation = terms_translation.apply(lambda x: x.str.replace('  ',' ').str.strip(' '))

In [120]:
AE['AEDECOD'] = AE.AEDECOD.apply(lambda x: x.replace('  ',' ').strip(' '))
AE['AEBODSYS'] = AE.AEBODSYS.apply(lambda x: x.replace('  ',' ').strip(' '))
AE['AEBODSYS']

0     General disorders and administration site cond...
1     General disorders and administration site cond...
2                           Renal and urinary disorders
3                           Renal and urinary disorders
4                           Renal and urinary disorders
5                           Renal and urinary disorders
6                           Renal and urinary disorders
7                                    Vascular disorders
8       Respiratory, thoracic and mediastinal disorders
9              Reproductive system and breast disorders
10                          Renal and urinary disorders
Name: AEBODSYS, dtype: object

In [121]:
AE_combined = AE.merge(terms_translation, left_on=['AEDECOD', 'AEBODSYS'], right_on=['PT', 'SOC'])
AE_combined['AEBODSYS_with_translation'] = AE_combined['AEBODSYS'] + ' (' + AE_combined['SOCT'] + ')'
adae['AEBODSYS']  = AE_combined['AEBODSYS_with_translation']
adae['AEDECOD'] = AE_combined['AEDECOD']

In [122]:
adae['AEBODSYS'].value_counts()

AEBODSYS_with_translation
Renal and urinary disorders (Нарушения со стороны почек и мочевыводящих путей)                                                      6
General disorders and administration site conditions (Общие нарушения и реакции в месте введения)                                   2
Vascular disorders (Нарушения со стороны сосудов)                                                                                   1
Respiratory, thoracic and mediastinal disorders (Нарушения со стороны дыхательной системы, органов грудной клетки и средостения)    1
Reproductive system and breast disorders (Нарушения со стороны репродуктивной системы и молочных желез)                             1
Name: count, dtype: int64

In [123]:
adae['AESTDTC'] = AE['AESTDTC'].dt.strftime('%Y-%m-%d')

for col in ['AP01SDT',
 'AP01EDT',
 'AP02SDT',
 'AP02EDT']:
     adae[col] = adae[col].dt.strftime('%d.%m.%Y')

In [124]:
adae['AEENDTC'] = AE['AEENDTC'].dt.strftime('%Y-%m-%d')
adae['AEENDTC']

0     2023-06-28
1     2023-06-29
2     2023-06-29
3     2023-06-29
4     2023-06-30
5     2023-07-05
6     2023-07-06
7     2023-07-06
8     2023-06-27
9     2023-06-28
10    2023-06-29
Name: AEENDTC, dtype: object

In [125]:
adae['AEENRTPT'] = AE['AEENRTPT']
adae['AEENRF'] = AE['AEENRTPT'].map(lambda x: 'ONGOING' if x == 'ONGOING' else np.nan)
adae['AEENRF'] 

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
Name: AEENRTPT, dtype: float64

In [126]:
adae['AESER'] = AE.AESER.map(lambda x:'N' if x=='Нет' else 'Y')

In [130]:
adae['ASEV'] = AE['AESEV'].map(lambda x:x[:-2]+'ая' if x=='Легкое' else "Средняя")
adae['ASEVN'] = adae['ASEV'].map(lambda x: 1 if x=='Легкая' else 2)

In [131]:
adae['AEREL'] = AE.AEREL

mapper = {
    'Определенная': 1,
    'Вероятная': 2,
    'Возможная': 3,
    'Сомнительная': 4,
    'Условная': 5,
    'Не классифицируемая': 6,
    'Не связано': 7
}

adae['AERELN'] = adae['AEREL'].map(lambda x: mapper[x])
adae['AERELN']

0     1
1     2
2     4
3     3
4     7
5     7
6     7
7     7
8     3
9     3
10    7
Name: AEREL, dtype: int64

In [132]:
mapper = {7:'Не связано',
         6: np.nan}
adae['RELGR1'] = adae['AERELN'].map(lambda x: mapper[x] if x in mapper.keys() else 'Связано')

In [133]:
mapper = {'Не связано':0,
          'Связано':1}
adae['RELGR1N'] = adae['RELGR1'].map(lambda x: mapper[x])
adae['RELGR1N']

0     1
1     1
2     1
3     1
4     0
5     0
6     0
7     0
8     1
9     1
10    0
Name: AEREL, dtype: int64

In [134]:
adae['AEACN'] = AE.AEACN
adae['AEACN']

0     Без изменений
1     Без изменений
2     Без изменений
3     Без изменений
4     Без изменений
5     Без изменений
6     Без изменений
7     Без изменений
8     Без изменений
9     Без изменений
10     Не применимо
Name: AEACN, dtype: object

In [135]:
adae['AERES']  = AE.AEOUT.map(lambda x: 'Выздоровление без осложнений' if x =='Выздоровление без последствий' else np.nan)
adae['AERES']

0     Выздоровление без осложнений
1     Выздоровление без осложнений
2     Выздоровление без осложнений
3     Выздоровление без осложнений
4     Выздоровление без осложнений
5     Выздоровление без осложнений
6     Выздоровление без осложнений
7     Выздоровление без осложнений
8     Выздоровление без осложнений
9     Выздоровление без осложнений
10    Выздоровление без осложнений
Name: AEOUT, dtype: object

In [136]:
adae['AERESN'] = adae['AERES'].map(lambda x: 1 if x == 'Выздоровление без осложнений' else np.nan)
adae['AERESN']

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
Name: AEOUT, dtype: int64

In [137]:
adae['AECMFL'] = AE.AECONTRT.map(lambda x:'N' if x=='Нет' else 'Y')
adae['AECMFL']

0     N
1     N
2     N
3     N
4     N
5     N
6     N
7     Y
8     N
9     N
10    N
Name: AECONTRT, dtype: object

In [138]:
adae['RACE']

0     Европеоидная
1     Европеоидная
2     Европеоидная
3     Европеоидная
4     Европеоидная
5     Европеоидная
6     Европеоидная
7     Европеоидная
8     Европеоидная
9     Европеоидная
10    Европеоидная
Name: SUBJID, dtype: object

In [139]:
adae['TRTEMFL'] = {}
combined = AE.merge(ADSL, on='SUBJID', how='left')
for i in combined.index:
    flag = 0
    d = combined.loc[i]
    ae_st = d.loc['AESTDTC']
    p1_st = d.loc['AP01SDT']
    p1_end = d.loc['AP01EDT']
    p2_st = d.loc['AP02SDT']
    p2_end = d.loc['AP02EDT']
    if p1_st and p1_end:
        if ae_st>=p1_st and ae_st<=p1_end:
            flag = 'Y'
        elif ae_st<p1_st:
            print(d.SUBJID)
            flag = None
    else:
        flag = None
    if p2_st and p2_end:
        if ae_st>=p2_st and ae_st<=p2_end:
            flag = 'Y'
    else:
        flag = None
    adae['TRTEMFL'][i] = flag
adae['TRTEMFL'] = pd.Series(adae['TRTEMFL'])
adae['TRTEMFL']

9008


0        Y
1        Y
2        Y
3        Y
4        Y
5        Y
6        Y
7        Y
8     None
9        Y
10       Y
dtype: object

In [140]:
adae['PREFL'] = {}
combined = AE.merge(ADSL, on='SUBJID', how='left')
for i in combined.index:
    flag = 0
    d = combined.loc[i]
    ae_st = d.loc['AESTDTC']
    p1_st = d.loc['AP01SDT']
    if p1_st:
        if ae_st<p1_st:
            print(d.SUBJID)
            flag = 'Y'
        else:
            flag = None
    adae['PREFL'][i] = flag
adae['PREFL'] = pd.Series(adae['PREFL'])
adae['PREFL']

9008


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8        Y
9     None
10    None
dtype: object

In [141]:
adae['AENDT']

0     28.06.2023
1     29.06.2023
2     29.06.2023
3     29.06.2023
4     30.06.2023
5     05.07.2023
6     06.07.2023
7     06.07.2023
8     27.06.2023
9     28.06.2023
10    29.06.2023
Name: AEENDTC, dtype: object

In [142]:
adae['ASTDTF'] = adae['ASTDT'].map(lambda x: np.nan)
adae['AENDTF'] = adae['AENDT'].map(lambda x: np.nan)

In [143]:
adae['ADURN'] = (AE.AESTDTC-AE.AEENDTC).dt.days+1
adae['ADURN']

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
dtype: int64

In [144]:
adae['ADURU'] = adae['ADURN'].map(lambda x:'день')
adae['ADURU']

0     день
1     день
2     день
3     день
4     день
5     день
6     день
7     день
8     день
9     день
10    день
dtype: object

In [145]:
adae['APHASE'] = {}

for i in adae['PREFL'].index:
    if adae['PREFL'].loc[i]=='Y':
        adae['APHASE'][i] = 'Скрининг'
    if adae['TRTEMFL'].loc[i] =='Y':
        adae['APHASE'][i] = 'Лечение'

adae['APHASE'] = pd.Series(adae['APHASE'])
adae['APHASE']

0      Лечение
1      Лечение
2      Лечение
3      Лечение
4      Лечение
5      Лечение
6      Лечение
7      Лечение
8     Скрининг
9      Лечение
10     Лечение
dtype: object

In [146]:
cols = input().split()

 STUDYID SUBJID USUBJID SITEID TRTSEQP TRTSEQPN AP01SDT AP01EDT AP02SDT AP02EDT AREPIOD APERIODC TRTEMFL PREFL TRTP TRTPN AESEQ  AETERM  AEBODSYS  AEDECOD AESTDTC ASTDT ASTDTF AEENDTC AENDT AENDTF AEENRTPT AEENRF ADURN ADURU AESER  APHASE  ASEV  ASEVN AEREL  AERELN RELGR1 RELGR1N AEACN AERES AERESN AECMFL SAFFL AGE SEX WEIGHT RACE


In [147]:
adae = pd.DataFrame(adae)

In [148]:
adae = adae[cols]

In [150]:
adae.to_excel('adae.xlsx', index=False)